In [1]:
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Conv2D , MaxPool2D, Flatten, Dense, Conv2D, Dropout, Convolution2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from keras.applications.vgg16 import VGG16

import warnings
import glob
import cv2

import numpy as np
from tensorflow.keras.preprocessing import image
warnings.filterwarnings('ignore')

In [2]:
DATASET_PATH = 'datasets/weapon_dataset_v2'

image_height = 128
image_width = 128
classes_list = [ 'normal', 'weapon']

In [3]:
vgg = VGG16(input_shape=[image_height, image_width] + [3], weights='imagenet', include_top=False)

In [4]:
for layer in vgg.layers:
    layer.trainable = False

In [5]:
x = Flatten()(vgg.output)

In [6]:
prediction = Dense(1, activation='sigmoid')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [8]:
# model = Sequential()
    
# # Layer: 1
# model.add(Conv2D(filters = 16, kernel_size = 3, padding='same',activation = 'relu', input_shape = (image_height, image_width, 3)))
# model.add(MaxPool2D(4,4))
# model.add(Dropout(0.25))

# # Layer: 2
# model.add(Conv2D(filters= 32, kernel_size=3,padding='same', activation='relu'))
# model.add(MaxPool2D(4,4))
# model.add(Dropout(0.25))

# # Layer: 3
# model.add(Conv2D(filters= 64, kernel_size=3,padding='same', activation='relu'))
# model.add(MaxPool2D(2,2))
# model.add(Dropout(0.25))

# # Layer: 4
# model.add(Conv2D(filters= 64, kernel_size=3,padding='same', activation='relu'))
# model.add(MaxPool2D(2,2))
# model.add(Dropout(0.25))

# # Flatten Layer
# model.add(Flatten())

# # Final Classification Layer
# model.add(Dense(1, activation='sigmoid'))

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
training_set = train_datagen.flow_from_directory(
                            DATASET_PATH + '/training_set',
                            target_size=(image_width, image_height),
                            class_mode='binary')

Found 354 images belonging to 2 classes.


In [11]:
test_set = train_datagen.flow_from_directory(
                            DATASET_PATH + '/testing_set',
                            target_size=(image_width, image_height),
                            class_mode='binary')

Found 58 images belonging to 2 classes.


In [12]:
# Adding Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
 
# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
 
# Start Training
model.fit_generator(training_set,
         steps_per_epoch=len(training_set),
         epochs=25,
         validation_data=test_set,
         validation_steps=len(test_set),
        callbacks = [early_stopping_callback])

Epoch 1/25
12/12 [==============================] - 66s 5s/step - loss: 0.4762 - accuracy: 0.7401 - val_loss: 0.4791 - val_accuracy: 0.8276
Epoch 2/25
12/12 [==============================] - 43s 3s/step - loss: 0.2495 - accuracy: 0.9124 - val_loss: 0.4215 - val_accuracy: 0.7931
Epoch 3/25
12/12 [==============================] - 43s 4s/step - loss: 0.1622 - accuracy: 0.9576 - val_loss: 0.3296 - val_accuracy: 0.8793
Epoch 4/25
12/12 [==============================] - 43s 4s/step - loss: 0.1613 - accuracy: 0.9463 - val_loss: 0.3848 - val_accuracy: 0.8276
Epoch 5/25
12/12 [==============================] - 46s 4s/step - loss: 0.1168 - accuracy: 0.9661 - val_loss: 0.2704 - val_accuracy: 0.8621
Epoch 6/25
12/12 [==============================] - 49s 4s/step - loss: 0.0939 - accuracy: 0.9802 - val_loss: 0.1823 - val_accuracy: 0.9310
Epoch 7/25
12/12 [==============================] - 45s 4s/step - loss: 0.0731 - accuracy: 0.9915 - val_loss: 0.2280 - val_accuracy: 0.8793
Epoch 8/25
12/12 [==

In [13]:
model.save('models/WEAPON_MODEL_4.h5')

In [14]:
test_image = image.load_img('testing_data/weapon.jpg', target_size=(image_width, image_height))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

prediction = model.predict(test_image)
print(prediction)

1/1 [==============================] - 0s 313ms/step
[[0.894908]]
